In [15]:
!wget -x --load-cookies cookies.txt "https://www.kaggle.com/c/13032/download-all" -O data.zip

--2020-01-26 11:10:27--  https://www.kaggle.com/c/13032/download-all
Resolving www.kaggle.com (www.kaggle.com)... 35.244.233.98
Connecting to www.kaggle.com (www.kaggle.com)|35.244.233.98|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/13032/862545/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1580296227&Signature=EZfzXgnmdJ%2FOAZxcocpc8qp4gEg09TwLpFpMG8pJ6vxZhUlPqYonKZurxjohSqXQc6x5kNhvL9J1p41UBauk3I97nYDrc7iCigVSVMV2YouNv9azPuEh6TZSsmR4ZfcezRahVuVEQCoDn42olsYW4SvohIu0QKHf59mc4oq14cBlLXd2VZGNOHPzI5AAhUUA7o4wH7Vo19u%2BxEB9%2BPWbIL7wIbOB%2Bsh3sjS8CWgCvH%2FRfptufq5QWI5M1CUy4bE%2B%2F7CxXFqxxdKN8F4rjKtwH7%2F1d6o%2FIYypHeyU7lHADDpnq9Jv%2BKEsMmdWe4lZZPjbH%2FO9fxM6j2qibSiv%2Fx9emA%3D%3D&response-content-disposition=attachment%3B+filename%3Dimaterialist-fashion-2019-FGVC6.zip [following]
--2020-01-26 11:10:27--  https://storage.googleapis.com/kaggle-compet

In [16]:
!unzip data.zip


  inflating: train/b33230ab935337acb514ca107487aad3.jpg  
  inflating: train/b332a0cf591dcca08d822a4118856698.jpg  
  inflating: train/b3337c9f7e35dd594f21e23ec29aa6af.jpg  
  inflating: train/b33719efba14b32136f4263169b28a6b.jpg  
  inflating: train/b33cd7b8b294a7bc53f610a4c710dcc6.jpg  
  inflating: train/b33cfec72bf935694c48546298227864.jpg  
  inflating: train/b33d621bd02f1e476d196bdbf5f160df.jpg  
  inflating: train/b3410ba3559f2eb3b0bdfc8c2b1cbb00.jpg  
  inflating: train/b3412395bf729329c33144bc55192ada.jpg  
  inflating: train/b34398dc30e1ee3a8ad5edd4824dfda8.jpg  
  inflating: train/b346146098dd9f8924b25ba04e5da13e.jpg  
  inflating: train/b348f39c3f2e34cb6eefc7f6810a880a.jpg  
  inflating: train/b34983ec390dbbeb53e0d4b0f467e40d.jpg  
  inflating: train/b349ba8a0c2189050435e430a407ea90.jpg  
  inflating: train/b34c3b928d73f826acce9e38e3ebfce8.jpg  
  inflating: train/b34d7aeb95debffb48f54babfaed0de9.jpg  
  inflating: train/b34dfb5d9b6b76629ad67aac79da8dac.jpg  
  inflating: 

In [0]:
!rm data.zip

In [17]:
import os
import gc
import sys
import json
import glob
import random

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import itertools

from imgaug import augmenters as iaa
from sklearn.model_selection import train_test_split

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


In [0]:
DATA_DIR = '/content'
ROOT_DIR = '/content'

NUM_CATS = 46
IMAGE_SIZE = 512

In [25]:
!git clone https://www.github.com/matterport/Mask_RCNN.git

Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 111.83 MiB | 44.09 MiB/s, done.
Resolving deltas: 100% (569/569), done.


In [28]:
sys.path.append(f'{ROOT_DIR}/Mask_RCNN')
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

Using TensorFlow backend.


In [29]:
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = 'mask_rcnn_coco.h5'

-rw-r--r-- 1 root root 246M Nov 26  2017 mask_rcnn_coco.h5


In [0]:
class FashionConfig(Config):
    NAME = "fashion"
    NUM_CLASSES = NUM_CATS + 1
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4
    
    BACKBONE = 'resnet50'
    
    IMAGE_MIN_DIM = IMAGE_SIZE
    IMAGE_MAX_DIM = IMAGE_SIZE    
    IMAGE_RESIZE_MODE = 'none'
    
    RPN_ANCHOR_SCALES = (16, 32, 64, 128, 256)
    
    # STEPS_PER_EPOCH should be the number of instances 
    # divided by (GPU_COUNT*IMAGES_PER_GPU), and so should VALIDATION_STEPS;
    # however, due to the time limit, I set them so that this kernel can be run in 9 hours
    STEPS_PER_EPOCH = 500
    VALIDATION_STEPS = 50
    
config = FashionConfig()

In [0]:
with open(f"{DATA_DIR}/label_descriptions.json") as f:
    label_descriptions = json.load(f)

label_names = [x['name'] for x in label_descriptions['categories']]

segment_df = pd.read_csv(f'{DATA_DIR}/train.csv')
images = os.listdir(f'{DATA_DIR}/train')
segment_df = segment_df.loc[segment_df['ImageId'].isin(images), :]
segment_df['CategoryId'] = segment_df['ClassId'].str.split('_').str[0]

image_df = segment_df.groupby('ImageId')['EncodedPixels', 'CategoryId'].agg(lambda x: list(x))
size_df = segment_df.groupby('ImageId')['Height', 'Width'].mean()
image_df = image_df.join(size_df, on='ImageId')

In [0]:
def resize_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE), interpolation=cv2.INTER_AREA)  
    return img

In [0]:
class FashionDataset(utils.Dataset):

    def __init__(self, df):
        super().__init__(self)
        
        for i, name in enumerate(label_names):
            self.add_class("fashion", i+1, name)
        
        for i, row in df.iterrows():
            self.add_image("fashion", 
                           image_id=row.name, 
                           path=f'{DATA_DIR}/train/{row.name}', 
                           labels=row['CategoryId'],
                           annotations=row['EncodedPixels'], 
                           height=row['Height'], width=row['Width'])

    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path'], [label_names[int(x)] for x in info['labels']]
    
    def load_image(self, image_id):
        return resize_image(self.image_info[image_id]['path'])

    def load_mask(self, image_id):
        images = self.image_info[image_id]
        mask = np.zeros((IMAGE_SIZE, IMAGE_SIZE, len(images['annotations'])))
        labels = []
        
        for i, (segmentation, label) in enumerate(zip(images['annotations'], images['labels'])):
            encoded_pixels = [int(x) for x in segmentation.split()]
            cur_mask = np.zeros(images['height'] * images['width'])

            for j, start_pixel in enumerate(encoded_pixels[::2]):
                cur_mask[start_pixel:start_pixel+encoded_pixels[2 * j + 1]] = 1

            cur_mask = cur_mask.reshape((images['height'], images['width']), order='F')
            mask[:, :, i] = cv2.resize(cur_mask, (IMAGE_SIZE, IMAGE_SIZE))
            labels.append(int(label) + 1)
            
        return mask, np.asarray(labels)

In [0]:
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(image_df, test_size=0.2)

train_dataset = FashionDataset(train_df)
valid_dataset = FashionDataset(valid_df)

train_dataset.prepare()
valid_dataset.prepare()

In [0]:
lr = 3e-4
epochs = 4

model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=['mrcnn_class_logits', 'mrcnn_bbox_fc', 'mrcnn_bbox', 'mrcnn_mask'])

augmentation = iaa.Sequential([
    iaa.Fliplr(0.3)
])

In [59]:
model.train(train_dataset, valid_dataset,
            learning_rate=lr, 
            epochs=epochs,
            layers='all',
            augmentation=augmentation)

history = model.keras_model.history.history


Starting at epoch 0. LR=0.0003

Checkpoint Path: /content/fashion20200126T1138/mask_rcnn_fashion_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3a_branch2b          (B

/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py:49: UserWarning: Using a generator with `use_multi

Epoch 1/4


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:709: UserWarning: An input could not be retrieved. It could be because a worker has died.We do not have any information on the lost sample.
  UserWarning)


 24/500 [>.............................] - ETA: 1:18:21 - loss: 5.0969 - rpn_class_loss: 0.2221 - rpn_bbox_loss: 1.5837 - mrcnn_class_loss: 1.5148 - mrcnn_bbox_loss: 0.9543 - mrcnn_mask_loss: 0.8219

Process ForkPoolWorker-13:
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 641, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "/content/Mask_RCNN/mrcnn/model.py", line 1709, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/content/Mask_RCNN/mrcnn/model.py", line 1283, in load_image_gt
    image_meta = compose_image_meta(image_id, original_shape, image.shape,
KeyboardInterrupt


KeyboardInterrupt: ignored

Process ForkPoolWorker-16:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", line 641, in next_sample
    return six.next(_SHARED_SEQUENCES[uid])
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py", lin